In [6]:
import keras 
from keras import activations, layers

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # here define the hyperparameter.
            units=hp.Int("units",min_value=32,max_value=512,step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10,activation="softmax"))
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],

    )
    return model

In [7]:
import keras_tuner

build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_1, built=False>

In [8]:

import keras 
from keras import activations, layers


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # here define the hyperparameter.
            units=hp.Int("units",min_value=32,max_value=512,step=32),
            # tune the activation function to use
            activation= hp.Choice("activation",["relu", "tanh"]), 
        )
    )
    # Tune whether to use the dropout
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10,activation="softmax"))
    # Define the optimizer learning rate as a hyperparameter
    learning_rate = hp.Float("lr",min_value=1e-4,max_value=1e-2,sampling="log")
    model.compile(
        optimizer= keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],

    )
    return model

build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_2, built=False>

In [9]:
hp = keras_tuner.HyperParameters()
print(hp.Int("units", min_value=32, max_value=512, step=32))

32


In [10]:

def call_existing_code(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model


build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_3, built=False>

In [11]:

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_4, built=False>

In [12]:


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())

    # 🔹 Tuner decide karega kitni layers
    num_layers = hp.Int("num_layers", min_value=3, max_value=10,step=1)

    for i in range(num_layers):
          model.add(
        layers.Dense(
            # here define the hyperparameter.
            units=hp.Int("units",min_value=32,max_value=512,step=32),
            # tune the activation function to use
            activation= hp.Choice("activation",["relu", "tanh"]), 
        )
    )
    # Tune whether to use the dropout
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
      
    model.add(layers.Dense(10,activation="softmax"))
    # Define the optimizer learning rate as a hyperparameter
    learning_rate = hp.Float("lr",min_value=1e-4,max_value=1e-2,sampling="log")
    model.compile(
        optimizer= keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],

    )
    return model

build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_5, built=False>

In [13]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [14]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 10, 'step': 1, 'sampling': 'linear'}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [15]:
import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [16]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 56s]
val_accuracy: 0.9493499994277954

Best val_accuracy So Far: 0.9519500136375427
Total elapsed time: 00h 01m 51s


In [19]:
# Get the top 1 models.
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 384)            │       301,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 384)            │       147,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 384)            │       147,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 384)            │       147,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         3,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 748,810 (2.86 MB)

 Trainable params: 748,810 (2.86 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
tuner.results_summary()

Results summary
Results in my_dir\helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 4
units: 384
activation: tanh
dropout: True
lr: 0.002143981088964242
Score: 0.9519500136375427

Trial 2 summary
Hyperparameters:
num_layers: 8
units: 480
activation: tanh
dropout: True
lr: 0.0008170324354132605
Score: 0.9493499994277954

Trial 1 summary
Hyperparameters:
num_layers: 7
units: 96
activation: tanh
dropout: False
lr: 0.007124507269165055
Score: 0.8971000015735626


In [22]:
# Build a Neural Network using Keras Tuner with the following conditions:
# 1. Input layer size = 784.
# 2. Hidden Layer 1: number of neurons should be tuned between 200 and 400.
# 3. Hidden Layer 2: number of neurons should be tuned between 100 and 200.
# 4. Add Batch Normalization after Hidden Layer 2 (fixed).
# 5. Hidden Layer 3: units and activation should be hyperparameters.
# 6. Hidden Layer 4: units and activation should be hyperparameters.
# 7. Add Batch Normalization after the variable hidden layers (optional – may or may not be added).
# 8. Output Layer: 10 neurons with softmax activation.
# 9. Learning rate should be tuned using log sampling.


In [32]:
import keras
import keras_tuner
from keras import layers, Input
from keras.layers import BatchNormalization
from keras.models import Sequential
from scipy.stats import monte_carlo_test

def my_build_model(hp):
    model = keras.Sequential()

    # Input Layer
    model.add(Input(shape=(784,)))
    #model.add(layers.Flatten(input_shape=(28,28))) --> ham aise bhi input le sakte h

    # Hidden Layer 1
    model.add(
        layers.Dense(
            units=hp.Int("Hidden1_units", 100, 400, step=20),
            activation=hp.Choice("hidden1_activation", ["relu", "tanh"])
        )
    )

    # Hidden Layer 2
    model.add(
        layers.Dense(
            units=hp.Int("Hidden2_units", 100, 200, step=20),
            activation=hp.Choice("hidden2_activation", ["relu", "tanh"])
        )
    )

    model.add(layers.BatchNormalization())

    # Additional Hidden Layers
    for i in range(hp.Int("num_layers", 2, 4)):
        model.add(
            layers.Dense(
                units=hp.Int("units_3", 50, 100, step=20),
                activation=hp.Choice("activation_3", ["relu", "tanh"]),
                kernel_regularizer=keras.regularizers.l2(
                    hp.Float("reg", 0.001, 0.1, sampling="log")
                )
            )
        )

    if hp.Boolean("batchnorm_after_h4"):
        model.add(layers.BatchNormalization())
    

    model.add(layers.Dense(10, activation="softmax"))

    learning_rate = hp.Float(
        "learning_rate",
        1e-4,
        1e-2,
        sampling="log"
    )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


In [33]:
tuner = keras_tuner.RandomSearch(
    hypermodel=my_build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="Build_In_Hypermodel",
)


In [38]:
import numpy as np
from tensorflow import keras

# Random input data (1000 samples, 784 features)
x_train = np.random.rand(1000, 784).astype("float32")

# Random labels (0-9)
y_train_labels = np.random.randint(0, 10, 1000)

# One-hot encoding
y_train = keras.utils.to_categorical(y_train_labels, 10)

# Validation data
x_val = np.random.rand(200, 784).astype("float32")
y_val_labels = np.random.randint(0, 10, 200)
y_val = keras.utils.to_categorical(y_val_labels, 10)

In [39]:
import keras
import keras_tuner
from keras import layers, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping


class MyHyperModel(keras_tuner.HyperModel):

    # ------------------------
    # BUILD SECTION
    # ------------------------
    def build(self, hp):

        model = keras.Sequential()

        # Input Layer
        model.add(Input(shape=(784,)))

        # Hidden Layer 1
        model.add(
            layers.Dense(
                units=hp.Int("Hidden1_units", 100, 400, step=20),
                activation=hp.Choice("hidden1_activation", ["relu", "tanh"])
            )
        )

        # Hidden Layer 2
        model.add(
            layers.Dense(
                units=hp.Int("Hidden2_units", 100, 200, step=20),
                activation=hp.Choice("hidden2_activation", ["relu", "tanh"])
            )
        )

        model.add(layers.BatchNormalization())

        # Additional Hidden Layers
        for i in range(hp.Int("num_layers", 2, 4)):
            model.add(
                layers.Dense(
                    units=hp.Int("units_3", 50, 100, step=20),
                    activation=hp.Choice("activation_3", ["relu", "tanh"]),
                    kernel_regularizer=keras.regularizers.l2(
                        hp.Float("reg", 0.001, 0.1, sampling="log")
                    )
                )
            )

        if hp.Boolean("batchnorm_after_h4"):
            model.add(layers.BatchNormalization())

        # Output Layer
        model.add(layers.Dense(10, activation="softmax"))

        # Learning Rate Tuning
        learning_rate = hp.Float(
            "learning_rate",
            1e-4,
            1e-2,
            sampling="log"
        )

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        return model

    # ------------------------
    # TRAINING TUNE SECTION
    # ------------------------
    def fit(self, hp, model, *args, **kwargs):

        # Tune EarlyStopping patience
        patience = hp.Int("patience", 3, 10)

        checkpoint_cb = ModelCheckpoint(
            "best_model.keras",
            monitor="val_loss",
            save_best_only=True
        )

        earlystop_cb = EarlyStopping(
            monitor="val_loss",
            patience=patience,
            restore_best_weights=True
        )

        # Preserve existing callbacks if passed
        callbacks = kwargs.pop("callbacks", [])
        callbacks.extend([checkpoint_cb, earlystop_cb])

        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [32, 64, 128]),
            shuffle=hp.Boolean("shuffle"),
            epochs=hp.Int("epochs", 10, 30, step=5),
            callbacks=callbacks,
            **kwargs
        )


# ------------------------
# TUNER SETUP
# ------------------------

tuner = keras_tuner.RandomSearch(
    hypermodel=MyHyperModel(),
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="My_project"
)


In [ ]:
import keras_tuner

hp = keras_tuner.HyperParameters()

hypermodel = MyHyperModel()   

model = hypermodel.build(hp)

hypermodel.fit(
    hp,
    model,
    x_train,
    y_train,
    validation_data=(x_val, y_val)
)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0990 - loss: 2.5922 - val_accuracy: 0.1000 - val_loss: 2.4449
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1290 - loss: 2.4860 - val_accuracy: 0.1100 - val_loss: 2.4398
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1570 - loss: 2.4176 - val_accuracy: 0.1100 - val_loss: 2.4323
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1830 - loss: 2.3628 - val_accuracy: 0.1050 - val_loss: 2.4284
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2190 - loss: 2.3126 - val_accuracy: 0.1150 - val_loss: 2.4256
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2420 - loss: 2.2643 - val_accuracy: 0.1050 - val_loss: 2.4276
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2690 - loss: 2.2210 - val_accuracy: 0.0950 - val_loss: 2.4314
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2950 - loss: 2.1783 - val_accuracy: 0.0950 - val_loss:

In [44]:
model.summary()


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_53 (Dense)                │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 290,932 (1.11 MB)

 Trainable params: 96,910 (378.55 KB)

 Non-trainable params: 200 (800.00 B)

 Optimizer params: 193,822 (757.12 KB)

In [50]:

# all code here ---------------------------------> 
import numpy as np
import keras
import keras_tuner
from keras import layers, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


# ======================================================
# RANDOM DATA (Since you said no data available)
# ======================================================

x_train = np.random.rand(1000, 784).astype("float32")
y_train_labels = np.random.randint(0, 10, 1000)
y_train = keras.utils.to_categorical(y_train_labels, 10)

x_val = np.random.rand(200, 784).astype("float32")
y_val_labels = np.random.randint(0, 10, 200)
y_val = keras.utils.to_categorical(y_val_labels, 10)

x_test = np.random.rand(200, 784).astype("float32")
y_test_labels = np.random.randint(0, 10, 200)
y_test = keras.utils.to_categorical(y_test_labels, 10)


# ======================================================
# HYPERMODEL CLASS
# ======================================================

class MyHyperModel(keras_tuner.HyperModel):

    def build(self, hp):

        model = keras.Sequential()
        model.add(Input(shape=(784,)))

        model.add(
            layers.Dense(
                units=hp.Int("Hidden1_units", 100, 400, step=20),
                activation=hp.Choice("hidden1_activation", ["relu", "tanh"])
            )
        )

        model.add(
            layers.Dense(
                units=hp.Int("Hidden2_units", 100, 200, step=20),
                activation=hp.Choice("hidden2_activation", ["relu", "tanh"])
            )
        )

        model.add(layers.BatchNormalization())

        for i in range(hp.Int("num_layers", 2, 4)):
            model.add(
                layers.Dense(
                    units=hp.Int("units_3", 50, 100, step=20),
                    activation=hp.Choice("activation_3", ["relu", "tanh"]),
                    kernel_regularizer=keras.regularizers.l2(
                        hp.Float("reg", 0.001, 0.1, sampling="log")
                    )
                )
            )

        if hp.Boolean("batchnorm_after_h4"):
            model.add(layers.BatchNormalization())

        model.add(layers.Dense(10, activation="softmax"))

        learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        return model

    def fit(self, hp, model, *args, **kwargs):

        patience = hp.Int("patience", 3, 10)

        checkpoint_cb = ModelCheckpoint(
            "best_model.keras",
            monitor="val_loss",
            save_best_only=True
        )

        earlystop_cb = EarlyStopping(
            monitor="val_loss",
            patience=patience,
            restore_best_weights=True
        )

        callbacks = kwargs.pop("callbacks", [])
        callbacks.extend([checkpoint_cb, earlystop_cb])

        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [32, 64, 128]),
            shuffle=hp.Boolean("shuffle"),
            epochs=hp.Int("epochs", 10, 30, step=5),
            callbacks=callbacks,
            **kwargs
        )


# ======================================================
# TUNER
# ======================================================

tuner = keras_tuner.RandomSearch(
    hypermodel=MyHyperModel(),
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="My_project"
)

# Run search
tuner.search(
    x_train,
    y_train,
    validation_data=(x_val, y_val)
)


# ======================================================
# BEST MODEL
# ======================================================

best_hp = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.hypermodel.build(best_hp)

# Train best model again (optional but recommended)
best_model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=20
)


# ======================================================
# EVALUATION
# ======================================================

test_loss, test_accuracy = best_model.evaluate(x_test, y_test)
print("\nTest Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Predictions
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\nClassification Report:\n")
print(classification_report(y_true_classes, y_pred_classes))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_true_classes, y_pred_classes))


Trial 5 Complete [00h 00m 10s]
val_accuracy: 0.15000000596046448

Best val_accuracy So Far: 0.15000000596046448
Total elapsed time: 00h 00m 42s
Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.1180 - loss: 7.7554 - val_accuracy: 0.0950 - val_loss: 7.5442
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1810 - loss: 7.3101 - val_accuracy: 0.0950 - val_loss: 7.2466
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2350 - loss: 6.9566 - val_accuracy: 0.1000 - val_loss: 6.9671
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3390 - loss: 6.6147 - val_accuracy: 0.1150 - val_loss: 6.7119
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4150 - loss: 6.2855 - val_accuracy: 0.1200 - val_loss: 6.4767
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5000 - loss: 5.9592 - val_accuracy: 0.0950 - val_loss: 6.2675
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5810 - loss: 5.6312 - val_accuracy:

In [52]:
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_53 (Dense)                │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 290,932 (1.11 MB)

 Trainable params: 96,910 (378.55 KB)

 Non-trainable params: 200 (800.00 B)

 Optimizer params: 193,822 (757.12 KB)

In [ ]:
# ref..

In [ ]:

class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        image_size = hp.Int("image_size", 10, 28)
        inputs = keras.Input(shape=(image_size, image_size))
        outputs = layers.Flatten()(inputs)
        outputs = layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )(outputs)
        outputs = layers.Dense(10, activation="softmax")(outputs)
        model = keras.Model(inputs, outputs)
        model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, x, y, validation_data=None, **kwargs):
        if hp.Boolean("normalize"):
            x = layers.Normalization()(x)
        image_size = hp.get("image_size")
        cropped_x = x[:, :image_size, :image_size, :]
        if validation_data:
            x_val, y_val = validation_data
            cropped_x_val = x_val[:, :image_size, :image_size, :]
            validation_data = (cropped_x_val, y_val)
        return model.fit(
            cropped_x,
            y,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            validation_data=validation_data,
            **kwargs,
        )


tuner = keras_tuner.RandomSearch(
    MyHyperModel(),
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="tune_hypermodel",
)

tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

In [ ]:
hypermodel = MyHyperModel()
best_hp = tuner.get_best_hyperparameters()[0]
model = hypermodel.build(best_hp)
hypermodel.fit(best_hp, model, x_all, y_all, epochs=1)